In [46]:

from DNA import DNA
from Graficador import *
from Seleccion import Seleccion
import pandas as pd
from Cromosoma import Cromosoma

In [47]:
DATOS_EN_TABLA = '../TablaCapitales.xlsx' 
RUTA =  DATOS_EN_TABLA
FILAS_TABLA = 24 
lista_provincias_visitadas = []
cantidad_km = 0 

df = pd.read_excel(DATOS_EN_TABLA)

def devolver_informacion_tabla_crudo():
    df = pd.read_excel(RUTA)
    tabla = df.values 
 
    return tabla

In [48]:
def procesar_informacion_tabla(tabla: pd.DataFrame):
    lista_provincias = list(tabla[:,0])
    
    return [lista_provincias, tabla]

lista_provincias,tabla = procesar_informacion_tabla(devolver_informacion_tabla_crudo())

In [49]:
 

def get_distance(city1: str, city2: str) -> float:
    # print(city1, city2)
    distancia = tabla[city1][city2+1]
    if type(distancia) == float:
        return distancia
    # return tabla[lista_provincias.index(city1)][lista_provincias.index(city2) +1]


Generando un marco de datos

In [50]:
datos = []


In [ ]:
obtener_mejor_cromosoma = lambda poblacion: sorted(poblacion, key=lambda cromosoma: cromosoma.fitness, reverse=True)[0]

In [51]:
def ciclo(poblacion,fObjetivo,fFitness,numero_ciclo):
    DNA1 =  DNA(poblacion)
    elites = []
    res_objetivo = DNA1.calcular_objetivo(fObjetivo)
    res_fitness = DNA1.calcular_fitness(fFitness)
 
    # print(res_objetivo)
    elites = Seleccion.elitismo(DNA1.poblacion,2) # NO ES UN METODO DE SELCCION
    cantidad = 50-len(elites)
    
    cromosomas_padres = Seleccion.torneo(DNA1.poblacion,cantidad)
    # cromosomas_padres = Seleccion.ruleta(DNA1.poblacion,vueltas=cantidad)
    cromosomas_hijos = DNA.cruzar_poblacion_numeros(cromosomas_padres)    

    mejor_cromosoma = obtener_mejor_cromosoma(poblacion)
    datos.append([numero_ciclo,mejor_cromosoma.genes,mejor_cromosoma.fitness,mejor_cromosoma.objetivo])

    if len(cromosomas_hijos)>0:
        nueva_poblacion = cromosomas_hijos + elites

    # print(len(nueva_poblacion))
    return nueva_poblacion


In [52]:
import pandas as pd
ciclos = 200
cantidad_cromosomas = 50
funcion_objetivo = lambda x,y: get_distance(x,y)       #  esto tiene que devolver la distancia, no?
fitness = lambda x: 1/x # distancia total?
Cromosoma.setProbMutacion(0.2)
Cromosoma.setProbCrossover(0.75)
poblacion = DNA.generar_poblacion_numeros(cantidad_cromosomas) 
mejor_cromosoma = [0,None] 

for i in range(ciclos):
    nueva_poblacion = ciclo(poblacion,funcion_objetivo,fitness,i)
    # for cromosoma in nueva_poblacion:
    if i == ciclos-1:
        for cromosoma in poblacion:  
            if cromosoma.fitness > mejor_cromosoma[0]:
                mejor_cromosoma[0] = cromosoma.objetivo
                mejor_cromosoma[1] = cromosoma
            
    
    poblacion = nueva_poblacion 

    
ultimo_cromosoma = mejor_cromosoma[1]
dist_total = 0
print(f'Ultimo cromosoma: {ultimo_cromosoma}')
for ciudad in range(0,len(ultimo_cromosoma.genes)-2):
    ciudad1 = ultimo_cromosoma.genes[ciudad]
    ciudad2 = ultimo_cromosoma.genes[ciudad+1]
    distancia = get_distance(ciudad1,ciudad2)
    dist_total += distancia
    print(f'{get_distance(ciudad1,ciudad2)}    | [{ciudad1}][{ciudad2}]  {lista_provincias[ciudad1]} -> {lista_provincias[ciudad2]} ')

print(f'KM(Totales = objt):  {ultimo_cromosoma.objetivo} km | fitness = {ultimo_cromosoma.fitness}') 


df = pd.DataFrame(datos, columns=['i', 'ruta', 'fitness', 'objetivo'])
print(df)



Ultimo cromosoma: [17, 6, 18, 1, 19, 10, 22, 12, 7, 20, 4, 0, 5, 13, 14, 16, 15, 21, 11, 2, 9, 8, 3] 11192.0
152.0    | [17][6]  San Juan -> Mendoza 
235.0    | [6][18]  Mendoza -> San Luis 
293.0    | [18][1]  San Luis -> Córdoba 
330.0    | [1][19]  Córdoba -> Santa Fe 
1349.0    | [19][10]  Santa Fe -> Rawson 
1300.0    | [10][22]  Rawson -> Ushuaia 
359.0    | [22][12]  Ushuaia -> Río Gallegos 
1410.0    | [12][7]  Río Gallegos -> Neuquén 
422.0    | [7][20]  Neuquén -> Santa Rosa 
602.0    | [20][4]  Santa Rosa -> La Plata 
53.0    | [4][0]  La Plata -> Cdad. de Bs. As. 
986.0    | [0][5]  Cdad. de Bs. As. -> La Rioja 
149.0    | [5][13]  La Rioja -> S.F.d.V.d. Catamarca 
189.0    | [13][14]  S.F.d.V.d. Catamarca -> S.M. de Tucumán 
228.0    | [14][16]  S.M. de Tucumán -> Salta 
67.0    | [16][15]  Salta -> S.S. de Jujuy 
414.0    | [15][21]  S.S. de Jujuy -> Sgo. Del Estero 
523.0    | [21][11]  Sgo. Del Estero -> Resistencia 
13.0    | [11][2]  Resistencia -> Corrientes 
291.0  